#  World Pop Data Download by certain country and time periods


## Requirements

In [1]:
pip install pystac-client requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [pystac-client]0m [pystac]
Note: you may need to restart the kernel to use updated packages.


In [2]:

import os, re, json, time
from pathlib import Path
import requests

## Data Retrieval

We retrieve populatution data for Algeria for time periods starting 2019 and ending at 2025

In [ ]:

YEARS = range(2019, 2026)          #Specify year
ISO3  = "DZA" #Specify country
OUT   = Path("worldpop_DZA_100m_2019_2025"); OUT.mkdir(exist_ok=True)

HDRS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) Colab/WorldPopDownloader",
    "Accept": "application/json, */*;q=0.8",
    "Connection": "keep-alive",
}

def build_direct_url(iso3, year, constrained=True):
    """Build direct WorldPop Global 2 URL"""
    iso_lower = iso3.lower()
    constraint = "CN" if constrained else "UC"
    constraint_dir = "constrained" if constrained else "unconstrained"

    return (f"https://data.worldpop.org/GIS/Population/Global_2015_2030/"
            f"R2025A/{year}/{iso3.upper()}/v1/100m/{constraint_dir}/"
            f"{iso_lower}_pop_{year}_{constraint}_100m_R2025A_v1.tif")

def download(url, dest, retries=3):
    for i in range(retries):
        try:
          #  print(f"  Downloading from: {url}")
            with requests.get(url, headers=HDRS, stream=True, timeout=180) as r:
                r.raise_for_status()
                total_size = int(r.headers.get('content-length', 0))
                downloaded = 0
                with open(dest, "wb") as f:
                    for chunk in r.iter_content(1024*1024):
                        if chunk:
                            f.write(chunk)
                            downloaded += len(chunk)
                            if total_size > 0:
                                pct = (downloaded / total_size) * 100
                            #    print(f"\r  Progress: {pct:.1f}%", end='')
           #     print()  
            return
        except Exception as e:
            if i == retries - 1:
                raise
            print(f"  Retry {i+1}/{retries} after error: {e}")
            time.sleep(2*(i+1))

failures = []
for y in YEARS:
    try:
        url = build_direct_url(ISO3, y, constrained=True)

        dest = OUT / f"dza_pop_{y}_CN_100m_R2025A_v1.tif"

        if dest.exists():
            size_mb = dest.stat().st_size / (1024**2)
            print(f"[skip] {y}: {dest.name} ({size_mb:.1f} MB)")
        else:
         #   print(f"[downloading] {y}:")
            download(url, dest)
            size_mb = dest.stat().st_size / (1024**2)
            print(f"[done] {y}: {dest.name} ({size_mb:.1f} MB)")
    except Exception as e:
        print(f"[fail] {y}: {e}")
        failures.append((y, str(e)))

print(f"\n{'='*60}")
if failures:
    for y, msg in failures:
        print(f"  {y}: {msg}")
else:
    print("All years completed successfully!")
    print(f"\nFiles saved to: {OUT}")

    # Show total size
    total_size = sum(f.stat().st_size for f in OUT.glob("*.tif"))
    print(f"Total size: {total_size / (1024**3):.2f} GB")
